In [1]:
#!pip install keras==2.12.0


In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import holidays

from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [157]:
from sklearn.model_selection import train_test_split, GridSearchCV, GridSearchCV, TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import Adam

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [156]:
from xgboost import XGBRegressor
from sklearn.metrics import make_scorer

In [6]:
#Set parameters to see all data
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [7]:
# Read dataset

%%time
data_mart_path = 'data_mart.csv'

try:
    data_mart_df = pd.read_csv(data_mart_path)
except pd.errors.ParserError as e:
    print(f'Error while parsing CSV file: {e}')



data_mart_df.shape

CPU times: user 711 ms, sys: 251 ms, total: 961 ms
Wall time: 965 ms


<timed exec>:4: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.


(101350, 104)

In [8]:
# Method for reducing the memory usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [9]:
data_mart_df = reduce_mem_usage(data_mart_df)

Mem. usage decreased to 37.89 Mb (52.9% reduction)


In [10]:
# Replace empty spaces of dataframe column names with  '_'
data_mart_df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
# Replace uppercase characters of dataframe column names with lowercase
data_mart_df.columns = data_mart_df.columns.str.lower()

In [11]:
# Create a dictionary to merge similar values
company_type_merge_dict = {
    'Limited (LTD)': 'Limited',
    'Partnership – 3 or less': 'Partnership',
    'Partnership – 4 or more': 'Partnership',
}

# Merge similar values using the dictionary
data_mart_df['company_type'] = data_mart_df['company_type'].replace(company_type_merge_dict)

# Display unique values
print(data_mart_df['company_type'].unique())


[nan 'Limited' 'Other' 'Public Limited Company (PLC)' 'Sole Trader'
 'Partnership' 'Council/Charity']


In [12]:
companycategory_merge_dict = {
    'PRI/LTD BY GUAR/NSC (Private, limited by guarantee, no share capital)': 'Private Limited Company',
    "PRI/LBG/NSC (Private, Limited by guarantee, no share capital, use of 'Limited' exemption)": 'Private Limited Company',
    'PRIV LTD SECT. 30 (Private limited company, section 30 of the Companies Act)': 'Private Limited Company'
}

# Merge similar values using the dictionary
data_mart_df['companycategory'] = data_mart_df['companycategory'].replace(companycategory_merge_dict)
# Display unique values
print(data_mart_df['company_type'].unique())


[nan 'Limited' 'Other' 'Public Limited Company (PLC)' 'Sole Trader'
 'Partnership' 'Council/Charity']


In [13]:
# Creating a dictionary to merge similar values
uk_region_merge_dict = {
    'East of England': 'East England',
    'East Midlands': 'East England',
}

# Merging similar values using the dictionary
data_mart_df['uk_region'] = data_mart_df['uk_region'].replace(uk_region_merge_dict)

In [14]:
# Creating a dictionary to merge similar values and shorten names
ruc11_merge_dict = {
    '3_Urban with Significant Rural (rural including hub towns 26-49%)': 'Urban with Significant Rural',
    '2_Largely Rural (rural including hub towns 50-79%) ': 'Largely Rural',
    '6_Urban with Major Conurbation': 'Urban with Major Conurbation',
    '4_Urban with City and Town': 'Urban with City and Town'
}

# Merging similar values and shortening names using the dictionary
data_mart_df['ruc11'] = data_mart_df['ruc11'].replace(ruc11_merge_dict)

In [15]:
# Check for missing values
missing_values = data_mart_df.isnull().sum()
missing_percentage = (missing_values / len(data_mart_df)) * 100
print(pd.concat([missing_values, missing_percentage], axis=1, keys=['Missing Values', 'Percentage']))

                                                    Missing Values  Percentage
toplevelcustomerid                                               0    0.000000
client_account_number                                            0    0.000000
credit_terms_requested                                       44300   43.709916
credit_terms_granted                                         61021   60.208189
company_type                                                 60666   59.857918
cash_flow_score                                              57022   56.262457
credit_limit_requested                                        6216    6.133202
credit_limit_assigned                                        19843   19.578688
num_employees                                                10569   10.428219
fleet_size_declared                                          58334   57.556981
account_source                                                3830    3.778984
lead_type                                           

In [16]:
# Create a list of columns with missing_percentage greater than 60% using the above code
high_missing_columns = missing_percentage[missing_percentage > 60].index.tolist()

# Display highly missing columns
print(len(high_missing_columns))
print(high_missing_columns)

32
['credit_terms_granted', 'lead_type', 'avg_retention_cases_per_year', 'avg_support_cases_per_year', 'avg_complaint_cases_per_year', 'promo_group', 'average_director_tenure', 'customertype', 'survey_year', 'survey_product_satisfaction', 'survey_support_satisfaction', 'survey_product_issue_last_6_months', 'survey_documentation_satisfaction', 'survey_valued_customer', 'survey_fuelcard_plus_expenses', 'survey_fleet_management', 'survey_national_fuel_network', 'survey_ev_payments', 'survey_fuel_discounts', 'survey_breakdown_cover', 'survey_parking', 'survey_online_account', 'survey_continue_using_product', 'survey_value_for_money', 'survey_comparison_to_other_companies', 'survey_hybrid_fleet_percentage', 'survey_ev_fleet_percentage', 'num_no_class_used', 'num_cars_used', 'num_lgv_used', 'num_hgv_used', 'num_total_vehicles_used']


In [18]:
# Drop negative values
data_mart_df = data_mart_df[(data_mart_df['total_spend_monthly_avg'] >= 0) &
                            (data_mart_df['total_transactions_monthly_avg'] >= 0) &
                            (data_mart_df['total_rev_monthly_avg'] >= 0)]

In [20]:
data_mart_df[['total_spend_monthly_avg', 'total_transactions_monthly_avg', 'total_rev_monthly_avg', 'churned']].head()

,total_spend_monthly_avg,total_transactions_monthly_avg,total_rev_monthly_avg,churned
0,55.0,1.0,4,0
1,92.0,2.0,3,0
2,12.0,0.0,1,0
3,223.0,3.0,4,0
4,87.0,1.0,8,0


In [21]:
# Check for missing values
CLV_missing_values = data_mart_df[['total_spend_monthly_avg', 'total_transactions_monthly_avg',
                               'total_rev_monthly_avg', 'churned']].isnull().sum()
CLV_missing_percentage = (CLV_missing_values / len(data_mart_df)) * 100
print(pd.concat([CLV_missing_values, CLV_missing_percentage], axis=1, keys=['Missing Values', 'Percentage']))

                                Missing Values  Percentage
total_spend_monthly_avg                      0         0.0
total_transactions_monthly_avg               0         0.0
total_rev_monthly_avg                        0         0.0
churned                                      0         0.0


In [22]:
data_mart_df_V2 = data_mart_df.copy()
data_mart_df_V2.shape

(79066, 104)

In [23]:
# Drop columns from the DataFrame
data_mart_df_V2.drop(columns=high_missing_columns, inplace=True)
print(data_mart_df_V2.shape)


(79066, 72)


In [24]:
cat_col_list = []
for col in data_mart_df_V2.select_dtypes(include=['object']).columns:
  #print(col, " : \n")
  if (data_mart_df[col].nunique() < 15):
    cat_col_list.append(col)
print(cat_col_list)

['company_type', 'domain_type', 'lineofbusiness', 'companycategory', 'uk_region', 'country', 'ruc11']


In [25]:
data_mart_df_V2.iloc[:, :35].head()

,toplevelcustomerid,client_account_number,credit_terms_requested,company_type,cash_flow_score,credit_limit_requested,credit_limit_assigned,num_employees,fleet_size_declared,account_source,domain_type,start_date,num_live_cards,fuel_usage_flag,ev_usage_flag,rev_toll_usage_flag,servicepointepyx_usage_flag,kwikfit_usage_flag,bmm_usage_flag,beyondfuel_usage_flag,commercialdelphicreditlimit,commercialdelphiscore,numberofccjsinlast2years,lineofbusiness,last_transaction_date,tenure,fuel_active,visa_active,ev_active,fuel_sites_monthly_avg,total_fee_charge_monthly_avg,number_of_distinct_fees_monthly_avg,number_of_web_users,number_of_logins_monthly_avg,siccode_sictext_1
0,100000,LAC29892001,NaN,NaN,NaN,0.0,NaN,NaN,NaN,3rd Party - Murco Local Account,personal,01/02/2001,2.0,1,0,0,0,0,0,0,1083.333374,75.3125,0.0,AllStar,02/02/2024,8400.0,1.0,0.0,0.0,1.0,17.0,2.0,NaN,NaN,NaN
1,100003,LAC29948001,NaN,NaN,NaN,0.0,NaN,3.0,NaN,3rd Party - Murco Local Account,commercial,06/02/2001,5.0,1,0,0,0,0,0,0,1250.000000,82.0000,0.0,AllStar,28/02/2024,8424.0,1.0,0.0,0.0,1.0,13.0,1.0,1.0,2.0,NaN
2,100004,LAC29949001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3rd Party - Murco Local Account,personal,06/02/2001,2.0,1,0,0,0,0,0,0,NaN,NaN,NaN,AllStar,12/02/2024,8408.0,1.0,0.0,0.0,1.0,5.0,1.0,1.0,2.0,NaN
3,100005,LAC29950001,NaN,NaN,NaN,0.0,NaN,1.0,NaN,3rd Party - Murco Local Account,personal,06/02/2001,12.0,1,0,0,0,0,0,0,906.250000,54.1250,0.0,AllStar,22/02/2024,8416.0,1.0,0.0,0.0,1.0,38.0,0.0,1.0,3.0,NaN
4,10002,28709954,NaN,NaN,NaN,0.0,NaN,4.0,NaN,Visa Upsell,personal,15/02/2004,1.0,1,0,0,0,0,0,0,8757.142578,83.3125,0.0,AllStar,26/02/2024,7316.0,1.0,0.0,0.0,1.0,27.0,4.0,1.0,2.0,NaN


In [26]:
data_mart_df_V2['start_date'] = pd.to_datetime(data_mart_df_V2['start_date'], format='%d/%m/%Y')

data_mart_df_V2.start_date.min(), data_mart_df_V2.start_date.max()

(Timestamp('1990-08-13 00:00:00'), Timestamp('2024-03-04 00:00:00'))

In [27]:
data_mart_df_V2['last_transaction_date'] = pd.to_datetime(data_mart_df_V2['last_transaction_date'], format='%d/%m/%Y')

data_mart_df_V2.last_transaction_date.min(), data_mart_df_V2.last_transaction_date.max()

(Timestamp('2019-10-06 00:00:00'), Timestamp('2024-03-15 00:00:00'))

In [28]:
# Date of The First Covid-19 Case and Brexit in the UK
pandemic_date = '31/01/2020'
brexit_date = '23/06/2016'

In [29]:
# Check data size of (pandemic) threshold days
print('Before 2000 => ', data_mart_df_V2[data_mart_df_V2.start_date < '01/01/2000'].shape[0])
print('Between 2000 and 2010 => ', data_mart_df_V2[(data_mart_df_V2.start_date >= '01/01/2000') & (data_mart_df_V2.start_date < '01/01/2010')].shape[0])
print('Between 2000 and Pandemic => ', data_mart_df_V2[(data_mart_df_V2.start_date >= '01/01/2000') & (data_mart_df_V2.start_date < pandemic_date)].shape[0])
print('After Pandemic => ', data_mart_df_V2[data_mart_df_V2.start_date >= pandemic_date].shape[0])


Before 2000 =>  3722
Between 2000 and 2010 =>  9989
Between 2000 and Pandemic =>  33731
After Pandemic =>  41613


In [30]:
# Check data size of (Brexit) threshold days
print('Before 2000 => ', data_mart_df_V2[data_mart_df_V2.start_date < '01/01/2000'].shape[0])
print('Between 2000 and 2010 => ', data_mart_df_V2[(data_mart_df_V2.start_date >= '01/01/2000') & (data_mart_df_V2.start_date < '01/01/2010')].shape[0])
print('Between 2000 and Brexit => ', data_mart_df_V2[(data_mart_df_V2.start_date >= '01/01/2000') & (data_mart_df_V2.start_date < brexit_date)].shape[0])
print('After Brexit => ', data_mart_df_V2[data_mart_df_V2.start_date >= brexit_date].shape[0])

Before 2000 =>  3722
Between 2000 and 2010 =>  9989
Between 2000 and Brexit =>  20413
After Brexit =>  54931


In [31]:
data_mart_df_V3 = data_mart_df_V2[data_mart_df_V2.start_date >= brexit_date].copy()
data_mart_df_V3.shape

(54931, 72)

In [32]:
# Check for missing values
CLV_missing_date_values = data_mart_df_V3[['start_date', 'last_transaction_date']].isnull().sum()
CLV_date_missing_percentage = (CLV_missing_date_values / len(data_mart_df_V3)) * 100
print(pd.concat([CLV_missing_date_values, CLV_date_missing_percentage], axis=1, keys=['Missing Values', 'Percentage']))

                       Missing Values  Percentage
start_date                          0         0.0
last_transaction_date               0         0.0


In [33]:
data_mart_df_V3[['start_date', 'last_transaction_date']].head()

,start_date,last_transaction_date
7492,2016-06-23,2023-03-28
7493,2016-06-23,2024-03-09
7495,2016-06-23,2021-12-07
7496,2016-06-23,2024-02-27
7497,2016-06-23,2024-02-28


In [34]:
data_mart_df_V3.start_date.min(), data_mart_df_V3.start_date.max()

(Timestamp('2016-06-23 00:00:00'), Timestamp('2024-03-04 00:00:00'))

In [35]:
data_mart_df_V3.last_transaction_date.min(), data_mart_df_V3.last_transaction_date.max()

(Timestamp('2020-07-20 00:00:00'), Timestamp('2024-03-15 00:00:00'))

In [36]:
data_mart_df_V3[data_mart_df_V3.last_transaction_date < '2024-03-09'].shape, data_mart_df_V3[data_mart_df_V3.last_transaction_date >= '2024-03-09'].shape

((43192, 72), (11739, 72))

In [37]:
# Load UK holidays
uk_holidays = holidays.UnitedKingdom()

# Function to check if a date is a holiday
def is_holiday(date_value):
    return date_value in uk_holidays

In [38]:
# Apply the function to create a holiday flag column
data_mart_df_V3['is_holiday'] = data_mart_df_V3['start_date'].apply(is_holiday).astype(int)

In [39]:
# Customer active days
data_mart_df_V3['customer_activity_duration'] = (data_mart_df_V3['last_transaction_date'] - data_mart_df_V3['start_date']).dt.days

In [40]:
# Calculating day difference for date column
def calculate_day_difference(df, column_name):
    df[column_name] = pd.to_datetime(df[column_name], format='%d/%m/%Y')#.dt.date
    df[f"{column_name}_diff"] = (df['last_transaction_date'].max() - df[column_name]).dt.days
    return df

In [41]:
data_mart_df_V3 = calculate_day_difference(data_mart_df_V3, 'start_date')
data_mart_df_V3 = calculate_day_difference(data_mart_df_V3, 'last_transaction_date')

data_mart_df_V3[['start_date', 'start_date_diff', 'last_transaction_date', 'last_transaction_date_diff']].head()

,start_date,start_date_diff,last_transaction_date,last_transaction_date_diff
7492,2016-06-23,2822,2023-03-28,353
7493,2016-06-23,2822,2024-03-09,6
7495,2016-06-23,2822,2021-12-07,829
7496,2016-06-23,2822,2024-02-27,17
7497,2016-06-23,2822,2024-02-28,16


In [42]:
# Calculate churn rate
churn_rate = data_mart_df_V3['churned'].value_counts(normalize=True) * 100
print("Churn Rate:\n", churn_rate)

Churn Rate:
 churned
0    51.2898
1    48.7102
Name: proportion, dtype: float64


In [43]:
# Customer Diversity
data_mart_df_V3['transaction_diversity'] = data_mart_df_V3[['fuel_usage_flag', 'ev_usage_flag', 'rev_toll_usage_flag']].sum(axis=1)
data_mart_df_V3[['transaction_diversity', 'fuel_usage_flag', 'ev_usage_flag', 'rev_toll_usage_flag']].head()


,transaction_diversity,fuel_usage_flag,ev_usage_flag,rev_toll_usage_flag
7492,1,1,0,0
7493,2,1,0,1
7495,1,1,0,0
7496,2,1,0,1
7497,1,1,0,0


In [44]:
data_mart_df_V3.iloc[:, :35].head()

,toplevelcustomerid,client_account_number,credit_terms_requested,company_type,cash_flow_score,credit_limit_requested,credit_limit_assigned,num_employees,fleet_size_declared,account_source,domain_type,start_date,num_live_cards,fuel_usage_flag,ev_usage_flag,rev_toll_usage_flag,servicepointepyx_usage_flag,kwikfit_usage_flag,bmm_usage_flag,beyondfuel_usage_flag,commercialdelphicreditlimit,commercialdelphiscore,numberofccjsinlast2years,lineofbusiness,last_transaction_date,tenure,fuel_active,visa_active,ev_active,fuel_sites_monthly_avg,total_fee_charge_monthly_avg,number_of_distinct_fees_monthly_avg,number_of_web_users,number_of_logins_monthly_avg,siccode_sictext_1
7492,132007,AS60126801,7+7,Limited,NaN,500.0,500.0,14.0,2.0,Direct - CFCUK,commercial,2016-06-23,1.0,1,0,0,0,0,0,0,6637.037109,65.8750,0.000000,AllStar,2023-03-28,2468.0,0.0,0.0,0.0,1.0,8.0,1.0,2.0,3.0,82990 - Other business support service activit...
7493,132008,AS60126802,7+7,Limited,87.0,2000.0,1600.0,4.0,6.0,Direct - FCUK,personal,2016-06-23,6.0,1,0,1,0,0,0,0,22605.554688,90.9375,0.000000,AllStar,2024-03-09,2816.0,1.0,0.0,0.0,15.0,113.0,2.0,1.0,4.0,43910 - Roofing activities
7495,132026,AS60126820,7+7,Limited,NaN,350.0,500.0,2.0,3.0,3rd Party - B&Q Tradepoint,commercial,2016-06-23,3.0,1,0,0,0,0,0,0,1325.000000,80.5625,0.000000,AllStar,2021-12-07,1993.0,0.0,0.0,0.0,2.0,13.0,1.0,1.0,3.0,NaN
7496,132038,AS60126832,7+7,Limited,NaN,1200.0,1000.0,4.0,NaN,Direct - ASUK,commercial,2016-06-23,7.0,1,0,1,0,0,0,0,5168.181641,82.8125,0.000000,AllStar,2024-02-27,2804.0,1.0,0.0,0.0,9.0,89.0,3.0,1.0,3.0,71129 - Other engineering activities
7497,132044,AS60126838,7+7,Limited,31.0,2050.0,1700.0,4.0,3.0,Direct - ASUK,personal,2016-06-23,9.0,1,0,0,0,0,0,0,11782.142578,69.0625,0.035706,AllStar,2024-02-28,2806.0,1.0,0.0,0.0,14.0,26.0,1.0,1.0,3.0,43341 - Painting


In [45]:
data_mart_df_V3.credit_terms_requested.unique()

array(['7+7', nan, '14+28', 'Other', 'Monthly+7', '14+14',
       'Best Available', '7+14', '14+7', 'Monthly + 30', 'Monthly + 14',
       '14+21', '7+Monthly', '7+21', '30+7', 'Monthly + 37',
       'Monthly + 15', 'Monthly + 28', 'Fortnightly + 10', 'Monthly + 21',
       'Weekly + 28', 'Weekly + 10', 'DA', 'Weekly + 30',
       'Fortnightly + 30', 'Monthly + 10', 'Weekly + 7', '7+5'],
      dtype=object)

In [46]:
# Custom encoding for 'credit_terms_requested'
def custom_encoding(term):
    if pd.isna(term):
        return 100  # Return 100 for NaN values

    # Replace date words with appropriate integers
    term = term.replace('Monthly', '30')
    term = term.replace('Weekly', '7')
    term = term.replace('Fortnightly', '14')
    term = term.replace('Others', '100')
    term = term.replace('Other', '100')
    term = term.replace('DA', '1')
    term = term.replace('Best Available', '0')

    # Split terms and calculate total days
    terms = term.split('+')
    days = sum(int(x) for x in terms)

    return days

In [47]:
data_mart_df_V3['credit_terms_requested'] = data_mart_df_V3['credit_terms_requested'].apply(custom_encoding)
data_mart_df_V3['credit_terms_requested'].unique()

array([ 14, 100,  42,  37,  28,   0,  21,  60,  44,  35,  67,  45,  58,
        24,  51,  17,   1,  40,  12])

In [48]:
data_mart_df_V3['company_type'].unique()


array(['Limited', 'Sole Trader', 'Other', nan, 'Partnership',
       'Council/Charity', 'Public Limited Company (PLC)'], dtype=object)

In [49]:
# Replace NaN with 'Other'
data_mart_df_V3['company_type'] = data_mart_df_V3['company_type'].fillna('Other')

# Replace white space with underscores
data_mart_df_V3['company_type'] = data_mart_df_V3['company_type'].str.replace(' ', '_')
data_mart_df_V3['company_type'] = data_mart_df_V3['company_type'].str.replace('/', '_')

# Replace 'Public Limited Company (PLC)' with 'PLC'
data_mart_df_V3['company_type'] = data_mart_df_V3['company_type'].replace('Public_Limited_Company_(PLC)', 'PLC')

# Display unique values after transformations
print(data_mart_df_V3['company_type'].unique())

['Limited' 'Sole_Trader' 'Other' 'Partnership' 'Council_Charity' 'PLC']


In [50]:
# Convert categorical variable into dummy/indicator variables
data_mart_df_V3 = pd.get_dummies(data_mart_df_V3, columns=['company_type'])

In [51]:
# Get columns containing 'life'
columns_containing_life = [col for col in data_mart_df_V3.columns if 'company_type' in col]
columns_containing_life

['company_type_Council_Charity',
 'company_type_Limited',
 'company_type_Other',
 'company_type_PLC',
 'company_type_Partnership',
 'company_type_Sole_Trader']

In [52]:
data_mart_df_V3.cash_flow_score.min(), data_mart_df_V3.cash_flow_score.mean(), data_mart_df_V3.cash_flow_score.max()

(0.0, inf, 400.0)

In [53]:
data_mart_df_V3 = reduce_mem_usage(data_mart_df_V3)

Mem. usage decreased to 15.35 Mb (11.7% reduction)


In [54]:
# Method for fill nan values with mean value of feature
def fill_nan_values_with_mean(df, feature):
  mean_score = round(df[feature].dropna().astype(np.int32).mean())
  df[feature] = df[feature].fillna(mean_score)
  return df

In [55]:
data_mart_df_V3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54931 entries, 7492 to 89407
Data columns (total 82 columns):
 #   Column                                                                                    Non-Null Count  Dtype         
---  ------                                                                                    --------------  -----         
 0   toplevelcustomerid                                                                        54931 non-null  int32         
 1   client_account_number                                                                     54931 non-null  object        
 2   credit_terms_requested                                                                    54931 non-null  int8          
 3   cash_flow_score                                                                           34435 non-null  float16       
 4   credit_limit_requested                                                                    54688 non-null  float32       
 5   credit

In [56]:
for col in data_mart_df_V3.select_dtypes(exclude=['object', 'bool', 'datetime64']).columns:
  print(col)
  data_mart_df_V3 = fill_nan_values_with_mean(data_mart_df_V3, col)


toplevelcustomerid
credit_terms_requested
cash_flow_score
credit_limit_requested
credit_limit_assigned
num_employees
fleet_size_declared
num_live_cards
fuel_usage_flag
ev_usage_flag
rev_toll_usage_flag
servicepointepyx_usage_flag
kwikfit_usage_flag
bmm_usage_flag
beyondfuel_usage_flag
commercialdelphicreditlimit
commercialdelphiscore
numberofccjsinlast2years
tenure
fuel_active
visa_active
ev_active
fuel_sites_monthly_avg
total_fee_charge_monthly_avg
number_of_distinct_fees_monthly_avg
number_of_web_users
number_of_logins_monthly_avg
mortgages_nummortcharges
age
sic_code
total_directors_over_time
average_age_of_directors
pence_per_litre
total_population_2011
rural_including_hub_towns_rural_&_rural_related_population_as_%_of_total_population_2011
ruc11cd
tenure_months
total_litres_monthly_avg
non_dd_litres_monthly_avg
dd_litres_monthly_avg
total_transactions_monthly_avg
total_transactions_fuel_monthly_avg
total_transactions_non_visa_monthly_avg
total_transactions_visa_monthly_avg
total_s

In [57]:
data_mart_df_V3.iloc[:, 35:].head()

,age,companycategory,sic_code,section,division,total_directors_over_time,average_age_of_directors,nationality,pence_per_litre,postcode_prefix,postcode_district,uk_region,country,county,total_population_2011,rural_including_hub_towns_rural_&_rural_related_population_as_%_of_total_population_2011,ruc11cd,ruc11,tenure_months,total_litres_monthly_avg,non_dd_litres_monthly_avg,dd_litres_monthly_avg,total_transactions_monthly_avg,total_transactions_fuel_monthly_avg,total_transactions_non_visa_monthly_avg,total_transactions_visa_monthly_avg,total_spend_monthly_avg,fuel_spend_monthly_avg,non_fuel_spend_non_visa_monthly_avg,non_fuel_spend_visa_monthly_avg,total_ev_transactions_monthly_avg,total_ev_energy_charge_per_kwh_monthly_avg,total_ev_session_charge_per_minute_monthly_avg,ev_spend_monthly_avg,total_rev_monthly_avg,churned,is_holiday,customer_activity_duration,start_date_diff,last_transaction_date_diff,transaction_diversity,company_type_Council_Charity,company_type_Limited,company_type_Other,company_type_PLC,company_type_Partnership,company_type_Sole_Trader
7492,9.0,Private Limited Company,82990.0,ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES,"Office administrative, office support and othe...",3.0,44.0,British,0.213135,TS,Cleveland,North East,England,north yorkshire,598376.0,73.125000,2.0,Largely Rural,82.0,69.0,69.0,0.0,1.0,1.0,0.0,0.0,102.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,15,1,0,2469,2822,353,1,False,True,False,False,False,False
7493,11.0,Private Limited Company,43910.0,CONSTRUCTION,Specialised construction activities,2.0,43.0,British,0.146729,SO,Southampton,South East,England,hampshire,1317788.0,29.484375,3.0,Urban with Significant Rural,94.0,1024.0,1020.0,4.0,18.0,17.0,0.0,0.0,1621.0,1600.0,0.0,0.0,0.0,0.0,0.0,0.0,150,0,0,2816,2822,6,2,False,True,False,False,False,False
7495,11.0,NaN,53672.0,NaN,NaN,3.0,48.0,NaN,0.156006,NaN,NaN,NaN,NaN,NaN,971888.0,47.000000,3.0,NaN,67.0,41.0,41.0,0.0,2.0,2.0,0.0,0.0,56.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,7,1,0,1993,2822,829,1,False,True,False,False,False,False
7496,8.0,Private Limited Company,71129.0,"PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES",Architectural and engineering activities; tech...,2.0,48.0,Burmese,0.231079,TW,Twickenham,Greater London,England,surrey,1132390.0,19.015625,6.0,Urban with Major Conurbation,93.0,372.0,372.0,1.0,9.0,9.0,0.0,0.0,593.0,592.0,0.0,0.0,0.0,0.0,0.0,0.0,86,0,0,2805,2822,17,2,False,True,False,False,False,False
7497,12.0,Private Limited Company,43341.0,CONSTRUCTION,Specialised construction activities,2.0,60.0,British,0.208130,SS,Southend,East England,England,essex,1393587.0,33.906250,3.0,Urban with Significant Rural,93.0,791.0,788.0,4.0,16.0,16.0,0.0,0.0,1305.0,1299.0,0.0,0.0,0.0,0.0,0.0,0.0,165,0,0,2806,2822,16,1,False,True,False,False,False,False


In [58]:
def replace_missing_with_other(data):
    # List of NaN, 'Unknown', and other possible missing values
    missing_values = [np.nan, 'Unknown', '0', 'Others', 'NaN', '']

    # Replace missing values with 'Other'
    data.replace(missing_values, 'Other', inplace=True)

    return data
for col in data_mart_df_V3.select_dtypes(include=['object']).columns:
  print(col)
  data_mart_df_V3[col] = replace_missing_with_other(data_mart_df_V3[col])

client_account_number
account_source
domain_type
lineofbusiness
siccode_sictext_1
companycategory
section
division
nationality
postcode_prefix
postcode_district
uk_region
country
county
ruc11


In [59]:
data_mart_df_V3.account_source.unique()

array(['Direct - CFCUK', 'Direct - FCUK', '3rd Party - B&Q Tradepoint',
       'Direct - ASUK', 'Partner - ECS', 'Partner - FSB',
       'Direct - Refer-a-Friend', 'Direct - Track Compare',
       'Direct - Facebook', '3rd Party - Lex Autolease', 'Data - IBEX',
       'Other', '3rd Party - AA', 'Direct - MVF',
       'Data - Experian Modelled', 'Keyfuels - Inbound',
       'Partner - Epyx DN', '3rd Party - ATS Assist',
       'Direct - pURL Referral', 'Direct - BFCUK',
       'Allstar - Network Service Fee Referral', 'Partner - QBE',
       'Data - Qbase', 'Allstar - Upsell', '3rd Party - RAC',
       'Data - Experian Pre Approved', 'Data - IBEX (Epyx Copy)',
       'Direct - Personal Lead', 'Winback', 'Direct - Lead Forensics',
       'Data - PreQualified Data (R)', 'Corp - New Unknown',
       'Direct - Direct Marketing', 'Direct - Exceptions', 'Internal',
       '3rd Party - PHVC', 'Data - Data', '3rd Party - Neva',
       '3rd Party - Asda LAC', 'Direct - Companeo',
       '3rd Par

In [60]:
data_mart_df_V3.domain_type.unique()

array(['commercial', 'personal', 'Other'], dtype=object)

In [61]:
data_mart_df_V3.lineofbusiness.unique()

array(['AllStar', 'Other'], dtype=object)

In [62]:
data_mart_df_V3['companycategory'] = data_mart_df_V3['companycategory'].str.replace(' ', '_')
data_mart_df_V3.companycategory.unique()

array(['Private_Limited_Company', 'Other', 'Community_Interest_Company',
       'Public_Limited_Company', 'Private_Unlimited_Company'],
      dtype=object)

In [63]:
data_mart_df_V3['uk_region'] = data_mart_df_V3['uk_region'].str.replace(' ', '_')
data_mart_df_V3.uk_region.unique()

array(['North_East', 'South_East', 'Other', 'Greater_London',
       'East_England', 'Wales', 'West_Midlands', 'North_West', 'Scotland',
       'South_West', 'Northern_Ireland', 'Non-geographic', 'Isle_of_Man'],
      dtype=object)

In [64]:
data_mart_df_V3['country'] = data_mart_df_V3['country'].str.replace(r' ', '_')

data_mart_df_V3.country.unique()

array(['England', 'Other', 'Wales', 'Scotland', 'Isle_of_Man'],
      dtype=object)

In [65]:
data_mart_df_V3['county'] = data_mart_df_V3['county'].str.replace(r' ', '_')

data_mart_df_V3.county.unique()

array(['north_yorkshire', 'hampshire', 'Other', 'surrey', 'essex',
       'hertfordshire', 'gloucestershire', 'worcestershire',
       'oxfordshire', 'suffolk', 'staffordshire', 'lancashire',
       'nottinghamshire', 'kent', 'warwickshire', 'west_sussex',
       'somerset', 'norfolk', 'cumbria', 'devon'], dtype=object)

In [66]:
data_mart_df_V3['ruc11'] = data_mart_df_V3['ruc11'].str.replace(r' ', '_')

data_mart_df_V3.ruc11.unique()

array(['Largely_Rural', 'Urban_with_Significant_Rural', 'Other',
       'Urban_with_Major_Conurbation', 'Urban_with_City_and_Town'],
      dtype=object)

In [67]:
# Remove leading or trailing spaces and other non-alphabetic characters
data_mart_df_V3['postcode_prefix'] = data_mart_df_V3['postcode_prefix'].str.replace(r'[^a-zA-Z]', '')

# Define a dictionary mapping similar values to a standardized representation
similar_values = {
    'Ts': 'TS',
    'mk': 'MK',
    'Ex': 'EX',
    'Le': 'LE',
    'Np': 'NP',
    'Yo': 'YO',
    'Bt': 'BT',
    'Hg': 'HG',
}

# Apply the mapping to merge similar values
data_mart_df_V3['postcode_prefix'] = data_mart_df_V3['postcode_prefix'].map(similar_values).fillna(data_mart_df_V3['postcode_prefix'])

# Remove duplicate values caused by the mapping
data_mart_df_V3['postcode_prefix'] = data_mart_df_V3['postcode_prefix'].str.upper()  # Convert to uppercase to ensure consistency
data_mart_df_V3['postcode_prefix'] = data_mart_df_V3['postcode_prefix'].drop_duplicates()

# Check unique values after corrections and merging
print(data_mart_df_V3['postcode_prefix'].unique())


['TS' 'SO' 'OTHER' 'TW' 'SS' 'W' nan 'CM' 'TN' 'NP' 'B' 'RG' 'NR' 'ST'
 'CF' 'DH' 'SW' 'SE' 'M' 'ML' 'WA' 'LU' 'DE' 'OL' 'NG' 'CT' 'EC' 'WF' 'ME'
 'TF' 'CV' 'E' 'FY' 'NN' 'DA' 'GU' 'CO' 'BN' 'MK' 'GL' 'CH' 'L' 'DD' 'HU'
 'BR' 'BH' 'IV' 'LS' 'BD' 'SK' 'DN' 'CR' 'AB' 'HP' 'SG' 'LE' 'WS' 'IP'
 'KT' 'DL' 'EN' 'EH' 'BS' 'PE' 'WD' 'DG' 'SL' 'CB' 'TQ' 'PL' 'KY' 'CA'
 'YO' 'DY' 'RM' 'WR' 'FK' 'HX' 'IG' 'HA' 'LL' 'SA' 'PO' 'BT' 'UB' 'G' 'SP'
 'NE' 'RH' 'S' 'HD' 'WC' 'BB' 'CW' 'EX' 'PH' 'DT' 'NW' 'WV' 'SR' 'BA' 'PR'
 'OX' 'AL' 'HG' 'LA' 'SY' 'KA' 'LN' 'N' 'WN' 'HR' 'PA' 'TR' 'TD' 'TA' 'BL'
 'SM' 'WG' 'HS' 'SN' 'KW' 'C' 'SSO ' 'LD' 'ZE' 'BX' 'IM' 'P' ' NP' 'HB'
 ': LE' 'AD' 'LG']


In [68]:
data_mart_df_V3.nationality.unique()

array(['British', 'Other', 'Burmese', 'English', 'Spanish', 'Dutch',
       'Irish', 'American', 'Swedish', 'South African', 'German',
       'United Kingdom', 'French', 'Singaporean', 'Pakistani', 'Italian',
       'Scottish', 'Canadian', 'Israeli', 'Swiss', 'Danish', 'Welsh',
       'Bulgarian', 'Chinese', 'New Zealander', 'Romanian', 'Australian',
       'Japanese', 'Norwegian', 'Polish', 'Portuguese', 'Cypriot',
       'Mauritian', 'Lithuanian', 'Hungarian', 'Indian', 'Austrian',
       'Czech', 'Greek', 'Ukrainian', 'Kenyan', 'Brazilian',
       'Albanian,British', 'Guyanese', 'Nepalese', 'German,British',
       'Nigerian', 'Albanian', 'Tanzanian', 'Northern Irish',
       'British,Portuguese', 'Turkish', 'Sri Lankan', 'Bangladeshi',
       'Latvian', 'British,', 'Ghanaian', 'Zimbabwean', 'Belgian',
       'British,Italian', 'Jordanian', 'South Korean', 'Slovak',
       'Croatian', 'Iraqi', 'Iranian', 'Russian', 'Cymraes', 'Malaysian',
       'Romanian,Moldovan', 'Cameroonian', '

In [69]:
# Define a dictionary mapping synonymous values to a standardized representation
synonymous_values = {
    'United Kingdom': 'British',
    'Brtish': 'British',
    'Brittish': 'British',
    'English': 'British',
    'Welsh': 'British',
    'Irish': 'British',  # Assuming Irish nationals living in the UK are included as British
    'Northern Irish': 'British',  # Assuming Northern Irish nationals are included as British
    'Cymraes': 'British',  # Assuming this refers to Welsh nationality
    'Scotland': 'British',  # Assuming Scottish nationals living in the UK are included as British
    'England': 'British',  # Assuming English nationals living in the UK are included as British
    'Irish,American': 'American, Irish',  # Assuming dual nationality
    'German,South African': 'German, South African'  # Assuming dual nationality
}

# Apply the mapping to merge similar values
data_mart_df_V3['nationality'] = data_mart_df_V3['nationality'].map(synonymous_values).fillna(data_mart_df_V3['nationality'])


In [70]:
def preprocess_text_similarity(text):
    # Tokenization
    tokens = word_tokenize(text)

    # Lowercasing and remove punctuation and special characters
    tokens = [re.sub(r'[^a-zA-Z0-9]', '', token.lower()) for token in tokens]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]

    # Join tokens back into a single string
    processed_text = ' '.join(tokens)

    return processed_text

In [71]:

data_mart_df_V3.section.unique()

array(['ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES', 'CONSTRUCTION',
       'Other', 'PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES',
       'WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES',
       'MANUFACTURING', 'HUMAN HEALTH AND SOCIAL WORK ACTIVITIES',
       'EDUCATION',
       'ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS; UNDIFFERENTIATED GOODS-AND SERVICES-PRODUCING ACTIVITIES OF HOUSEHOLDS FOR OWN USE',
       'INFORMATION AND COMMUNICATION', 'REAL ESTATE ACTIVITIES',
       'WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES',
       'ARTS, ENTERTAINMENT AND RECREATION', 'TRANSPORTATION AND STORAGE',
       'ACCOMMODATION AND FOOD SERVICE ACTIVITIES',
       'OTHER SERVICE ACTIVITIES', 'AGRICULTURE, FORESTRY AND FISHING',
       'ACTIVITIES OF EXTRATERRITORIAL ORGANISATIONS AND BODIES',
       'FINANCIAL AND INSURANCE ACTIVITIES',
       'PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY',
       'MINING AND QUARRYING',
    

In [72]:
# Apply the preprocessing function to the 'ruc11' column
data_mart_df_V3['processed_ruc11'] = data_mart_df_V3['ruc11'].apply(preprocess_text_similarity)
data_mart_df_V3.processed_ruc11.unique()

array(['largelyrur', 'urbanwithsignificantrur', '',
       'urbanwithmajorconurb', 'urbanwithcityandtown'], dtype=object)

In [73]:
# Apply the preprocessing function to the 'section' column
data_mart_df_V3['processed_section'] = data_mart_df_V3['section'].apply(preprocess_text_similarity)
data_mart_df_V3.processed_section.unique()

array(['administr support servic activ', 'construct', '',
       'profession  scientif technic activ',
       'wholesal retail trade  repair motor vehicl motorcycl',
       'manufactur', 'human health social work activ', 'educ',
       'activ household employ  undifferenti goodsand servicesproduc activ household use',
       'inform commun', 'real estat activ',
       'water suppli  sewerag  wast manag remedi activ',
       'art  entertain recreat', 'transport storag',
       'accommod food servic activ', 'servic activ',
       'agricultur  forestri fish', 'activ extraterritori organis bodi',
       'financi insur activ',
       'public administr defenc  compulsori social secur', 'mine quarri',
       'electr  ga  steam air condit suppli'], dtype=object)

In [74]:
# Apply the preprocessing function to the 'uk_region' column
data_mart_df_V3['processed_uk_region'] = data_mart_df_V3['uk_region'].apply(preprocess_text_similarity)
data_mart_df_V3.processed_uk_region.unique()

array(['northeast', 'southeast', '', 'greaterlondon', 'eastengland',
       'wale', 'westmidland', 'northwest', 'scotland', 'southwest',
       'northernireland', 'nongeograph', 'isleofman'], dtype=object)

In [75]:

# Apply the preprocessing function to the 'division' column
data_mart_df_V3['processed_division'] = data_mart_df_V3['division'].apply(preprocess_text_similarity)
data_mart_df_V3.processed_division.unique()

array(['offic administr  offic support busi support activ',
       'specialis construct activ', '',
       'architectur engin activ  technic test analysi',
       'wholesal trade  except motor vehicl motorcycl',
       'manufactur basic metal',
       'wholesal retail trade repair motor vehicl motorcycl',
       'profession  scientif technic activ', 'manufactur electr equip',
       'manufactur fabric metal product  except machineri equip',
       'servic build landscap activ', 'manufactur nonmetal miner product',
       'manufactur', 'repair instal machineri equip', 'construct build',
       'manufactur textil', 'residenti care activ', 'educ',
       'activ head offic  manag consult activ',
       'activ household employ domest personnel',
       'retail trade  except motor vehicl motorcycl',
       'manufactur machineri equip nec ',
       'motion pictur  video televis programm product  sound record music publish activ',
       'real estat activ',
       'wast collect  treatment disp

In [76]:
# LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder
data_mart_df_V3.account_source = label_encoder.fit_transform(data_mart_df_V3.account_source)
data_mart_df_V3.siccode_sictext_1 = label_encoder.fit_transform(data_mart_df_V3.siccode_sictext_1)
data_mart_df_V3.nationality = label_encoder.fit_transform(data_mart_df_V3.nationality)
data_mart_df_V3.processed_section = label_encoder.fit_transform(data_mart_df_V3.processed_section)
data_mart_df_V3.processed_division = label_encoder.fit_transform(data_mart_df_V3.processed_division)
data_mart_df_V3.processed_uk_region = label_encoder.fit_transform(data_mart_df_V3.processed_uk_region)
data_mart_df_V3.processed_ruc11 = label_encoder.fit_transform(data_mart_df_V3.processed_ruc11)
data_mart_df_V3.postcode_prefix = label_encoder.fit_transform(data_mart_df_V3.postcode_prefix)


In [77]:
get_dummy_cols = ['domain_type', 'lineofbusiness', 'companycategory', 'uk_region', 'country', 'county', 'ruc11']
# Convert categorical variable into dummy/indicator variables
for col in get_dummy_cols:
  data_mart_df_V3 = pd.concat([data_mart_df_V3, pd.get_dummies(data_mart_df_V3[col],
                                                               prefix=f"{col}_cat")], axis=1)


# Now drop the original categorical columns
data_mart_df_V3.drop(get_dummy_cols, axis=1, inplace=True)

In [78]:
data_mart_df_V3.shape

(54931, 132)

In [79]:
drop_cols = ['postcode_district',
             'account_source', 'siccode_sictext_1', 'nationality',
             'section', 'division', 'postcode_prefix']
# Drop columns from the DataFrame
data_mart_df_V3.drop(columns=drop_cols, inplace=True)

In [80]:
data_mart_df_V3.iloc[:, 80:].head()

,companycategory_cat_Private_Unlimited_Company,companycategory_cat_Public_Limited_Company,uk_region_cat_East_England,uk_region_cat_Greater_London,uk_region_cat_Isle_of_Man,uk_region_cat_Non-geographic,uk_region_cat_North_East,uk_region_cat_North_West,uk_region_cat_Northern_Ireland,uk_region_cat_Other,uk_region_cat_Scotland,uk_region_cat_South_East,uk_region_cat_South_West,uk_region_cat_Wales,uk_region_cat_West_Midlands,country_cat_England,country_cat_Isle_of_Man,country_cat_Other,country_cat_Scotland,country_cat_Wales,county_cat_Other,county_cat_cumbria,county_cat_devon,county_cat_essex,county_cat_gloucestershire,county_cat_hampshire,county_cat_hertfordshire,county_cat_kent,county_cat_lancashire,county_cat_norfolk,county_cat_north_yorkshire,county_cat_nottinghamshire,county_cat_oxfordshire,county_cat_somerset,county_cat_staffordshire,county_cat_suffolk,county_cat_surrey,county_cat_warwickshire,county_cat_west_sussex,county_cat_worcestershire,ruc11_cat_Largely_Rural,ruc11_cat_Other,ruc11_cat_Urban_with_City_and_Town,ruc11_cat_Urban_with_Major_Conurbation,ruc11_cat_Urban_with_Significant_Rural
7492,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
7493,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
7495,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False
7496,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,False
7497,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True


In [81]:
data_mart_df_V3.iloc[:, 80:].info()

<class 'pandas.core.frame.DataFrame'>
Index: 54931 entries, 7492 to 89407
Data columns (total 45 columns):
 #   Column                                         Non-Null Count  Dtype
---  ------                                         --------------  -----
 0   companycategory_cat_Private_Unlimited_Company  54931 non-null  bool 
 1   companycategory_cat_Public_Limited_Company     54931 non-null  bool 
 2   uk_region_cat_East_England                     54931 non-null  bool 
 3   uk_region_cat_Greater_London                   54931 non-null  bool 
 4   uk_region_cat_Isle_of_Man                      54931 non-null  bool 
 5   uk_region_cat_Non-geographic                   54931 non-null  bool 
 6   uk_region_cat_North_East                       54931 non-null  bool 
 7   uk_region_cat_North_West                       54931 non-null  bool 
 8   uk_region_cat_Northern_Ireland                 54931 non-null  bool 
 9   uk_region_cat_Other                            54931 non-null  bool 
 10  

In [82]:
# Replace boolean values in DataFrame with numeric equivalents.

def replace_boolean_with_numeric(df):

    # Select boolean columns
    boolean_cols = df.select_dtypes(include=bool).columns

    # Iterate over boolean columns and replace False with 0 and True with 1
    for col in boolean_cols:
        df[col] = df[col].astype(int)

    return df

In [83]:
data_mart_df_V4 = replace_boolean_with_numeric(data_mart_df_V3)
data_mart_df_V4.iloc[:, 80:].head()

,companycategory_cat_Private_Unlimited_Company,companycategory_cat_Public_Limited_Company,uk_region_cat_East_England,uk_region_cat_Greater_London,uk_region_cat_Isle_of_Man,uk_region_cat_Non-geographic,uk_region_cat_North_East,uk_region_cat_North_West,uk_region_cat_Northern_Ireland,uk_region_cat_Other,uk_region_cat_Scotland,uk_region_cat_South_East,uk_region_cat_South_West,uk_region_cat_Wales,uk_region_cat_West_Midlands,country_cat_England,country_cat_Isle_of_Man,country_cat_Other,country_cat_Scotland,country_cat_Wales,county_cat_Other,county_cat_cumbria,county_cat_devon,county_cat_essex,county_cat_gloucestershire,county_cat_hampshire,county_cat_hertfordshire,county_cat_kent,county_cat_lancashire,county_cat_norfolk,county_cat_north_yorkshire,county_cat_nottinghamshire,county_cat_oxfordshire,county_cat_somerset,county_cat_staffordshire,county_cat_suffolk,county_cat_surrey,county_cat_warwickshire,county_cat_west_sussex,county_cat_worcestershire,ruc11_cat_Largely_Rural,ruc11_cat_Other,ruc11_cat_Urban_with_City_and_Town,ruc11_cat_Urban_with_Major_Conurbation,ruc11_cat_Urban_with_Significant_Rural
7492,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
7493,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7495,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
7496,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
7497,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [84]:
data_mart_df_V4 = reduce_mem_usage(data_mart_df_V4)

Mem. usage decreased to 12.47 Mb (64.9% reduction)


In [85]:
# Rename rural_including_hub_towns_rural_&_rural_related_population_as_%_of_total_population_2011
data_mart_df_V4.rename(columns={'rural_including_hub_towns_rural_&_rural_related_population_as_%_of_total_population_2011': 'rural_population_2011'}, inplace=True)


In [86]:
data_mart_df_V4.iloc[:, 40:80].head()

,total_litres_monthly_avg,non_dd_litres_monthly_avg,dd_litres_monthly_avg,total_transactions_monthly_avg,total_transactions_fuel_monthly_avg,total_transactions_non_visa_monthly_avg,total_transactions_visa_monthly_avg,total_spend_monthly_avg,fuel_spend_monthly_avg,non_fuel_spend_non_visa_monthly_avg,non_fuel_spend_visa_monthly_avg,total_ev_transactions_monthly_avg,total_ev_energy_charge_per_kwh_monthly_avg,total_ev_session_charge_per_minute_monthly_avg,ev_spend_monthly_avg,total_rev_monthly_avg,churned,is_holiday,customer_activity_duration,start_date_diff,last_transaction_date_diff,transaction_diversity,company_type_Council_Charity,company_type_Limited,company_type_Other,company_type_PLC,company_type_Partnership,company_type_Sole_Trader,processed_ruc11,processed_section,processed_uk_region,processed_division,domain_type_cat_Other,domain_type_cat_commercial,domain_type_cat_personal,lineofbusiness_cat_AllStar,lineofbusiness_cat_Other,companycategory_cat_Community_Interest_Company,companycategory_cat_Other,companycategory_cat_Private_Limited_Company
7492,69.0,69.0,0.0,1.0,1.0,0.0,0.0,102.0,102.0,0.0,0.0,0.0,0.0,0.0,0.0,15,1,0,2469,2822,353,1,0,1,0,0,0,0,1,4,5,57,0,1,0,1,0,0,0,1
7493,1024.0,1020.0,4.0,18.0,17.0,0.0,0.0,1621.0,1600.0,0.0,0.0,0.0,0.0,0.0,0.0,150,0,0,2816,2822,6,2,0,1,0,0,0,0,4,7,9,77,0,0,1,1,0,0,0,1
7495,41.0,41.0,0.0,2.0,2.0,0.0,0.0,56.0,56.0,0.0,0.0,0.0,0.0,0.0,0.0,7,1,0,1993,2822,829,1,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0
7496,372.0,372.0,1.0,9.0,9.0,0.0,0.0,593.0,592.0,0.0,0.0,0.0,0.0,0.0,0.0,86,0,0,2805,2822,17,2,0,1,0,0,0,0,3,15,2,9,0,1,0,1,0,0,0,1
7497,791.0,788.0,4.0,16.0,16.0,0.0,0.0,1305.0,1299.0,0.0,0.0,0.0,0.0,0.0,0.0,165,0,0,2806,2822,16,1,0,1,0,0,0,0,4,7,1,77,0,0,1,1,0,0,0,1


In [87]:
# Do not use these columns to train the model
do_not_use_cols = ['total_spend_monthly_avg',
                   'total_transactions_monthly_avg',
                   'churned']

In [88]:
# Set numeric groups
get_bins_cols = ['num_employees', 'fleet_size_declared', 'number_of_web_users', 'number_of_logins_monthly_avg']
get_quarter_cols = ['credit_terms_requested', 'num_live_cards', 'mortgages_nummortcharges',
                    'total_directors_over_time','total_population_2011', 'rural_population_2011']
get_scaler_cols = ['cash_flow_score', 'credit_limit_requested', 'credit_limit_assigned', 'transaction_diversity',
                   'start_date_diff', 'last_transaction_date_diff', 'customer_activity_duration', 'tenure',
                   'commercialdelphicreditlimit', 'commercialdelphiscore',
                   'fuel_sites_monthly_avg', 'total_fee_charge_monthly_avg', 'number_of_distinct_fees_monthly_avg',
                   'sic_code', 'pence_per_litre', 'ruc11cd',  'tenure_months', 'age',  'average_age_of_directors',
                   'total_litres_monthly_avg', 'non_dd_litres_monthly_avg', 'dd_litres_monthly_avg',
                   'total_transactions_fuel_monthly_avg', 'total_transactions_non_visa_monthly_avg',
                   'total_transactions_visa_monthly_avg', 'fuel_spend_monthly_avg',
                   'non_fuel_spend_non_visa_monthly_avg', 'non_fuel_spend_visa_monthly_avg',
                   'total_ev_transactions_monthly_avg', 'total_ev_energy_charge_per_kwh_monthly_avg',
                   'total_ev_session_charge_per_minute_monthly_avg', 'ev_spend_monthly_avg',
                   'processed_ruc11',	'processed_section',	'processed_uk_region',	'processed_division']
manuel_scaler = ['numberofccjsinlast2years']

In [89]:
# Create bins for 'num_employees'
bins = [data_mart_df_V4['num_employees'].min(),
        0,
        data_mart_df_V4['num_employees'].astype(int).mean(),
        data_mart_df_V4['num_employees'].astype(int).mean() + data_mart_df_V4['num_employees'].astype(int).std(),
        data_mart_df_V4['num_employees'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V4['num_employees'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V4.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
          num_employees  count
0           (-1.0, 0.0]   4977
1         (0.0, 64.591]  43754
2    (64.591, 2100.462]   6063
3  (2100.462, 250000.0]    130


In [90]:
# Create bins for 'fleet_size_declared'
bins = [data_mart_df_V4['fleet_size_declared'].min(),
        data_mart_df_V4['fleet_size_declared'].astype(int).mean(),
        data_mart_df_V4['fleet_size_declared'].astype(int).mean() + data_mart_df_V4['fleet_size_declared'].astype(int).std(),
        data_mart_df_V4['fleet_size_declared'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V4['fleet_size_declared'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V4.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
    fleet_size_declared  count
0         (1.0, 35.994]  14517
1    (35.994, 1072.528]  33712
2  (1072.528, 200000.0]     68


In [91]:
# Create bins for 'number_of_web_users'
bins = [data_mart_df_V4['number_of_web_users'].min(),
        data_mart_df_V4['number_of_web_users'].astype(int).mean(),
        data_mart_df_V4['number_of_web_users'].astype(int).mean() + data_mart_df_V4['number_of_web_users'].astype(int).std(),
        data_mart_df_V4['number_of_web_users'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V4['number_of_web_users'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V4.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
  number_of_web_users  count
0        (1.0, 2.146]   7938
1     (2.146, 47.385]   6165
2    (47.385, 9496.0]     75


In [92]:
#number_of_logins_monthly_avg
# Create bins for 'number_of_logins_monthly_avg'
bins = [data_mart_df_V4['number_of_logins_monthly_avg'].min(),
        data_mart_df_V4['number_of_logins_monthly_avg'].astype(int).mean(),
        data_mart_df_V4['number_of_logins_monthly_avg'].astype(int).mean() + data_mart_df_V4['number_of_logins_monthly_avg'].astype(int).std(),
        data_mart_df_V4['number_of_logins_monthly_avg'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V4['number_of_logins_monthly_avg'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V4.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
  number_of_logins_monthly_avg  count
0                 (1.0, 2.636]  29695
1               (2.636, 5.635]  24366
2               (5.635, 410.0]    803


In [93]:
# Replace numeric columns in the dataframe with bin codes based on specified bin ranges.

def replace_columns_with_bins(dataframe, column_list):

    for column_name in column_list:
        print(column_name)
        # Check if the column exists and is numeric
        if column_name in dataframe.columns and pd.api.types.is_numeric_dtype(dataframe[column_name]):
            # Define bin ranges based on min, mean, mean+std, and max values
            bin_ranges = [dataframe[column_name].min(),
                          dataframe[column_name].astype(int).mean(),
                          dataframe[column_name].astype(int).mean() + dataframe[column_name].astype(int).std(),
                          dataframe[column_name].max()]
            # Replace the numeric values with bin codes
            dataframe[column_name] = pd.cut(dataframe[column_name],
                                            bins=bin_ranges,
                                            include_lowest=True,
                                            duplicates='drop').cat.codes
    return dataframe


In [94]:
data_mart_df_V5 = replace_columns_with_bins(data_mart_df_V4, get_bins_cols)
data_mart_df_V5[get_bins_cols].head()

num_employees
fleet_size_declared
number_of_web_users
number_of_logins_monthly_avg


,num_employees,fleet_size_declared,number_of_web_users,number_of_logins_monthly_avg
7492,0,0,0,1
7493,0,0,0,1
7495,0,0,0,1
7496,0,1,0,1
7497,0,0,0,1


In [95]:
# Convert a numerical column in a DataFrame to quartiles.
def convert_to_quartiles(dataframe, column_name_list):
    for col in column_name_list:
        quartiles = np.percentile(dataframe[col], [0, 25, 50, 75, 100])
        # Add a small epsilon value to ensure uniqueness
        epsilon = 1e-10
        quartiles += epsilon * np.arange(5)
        dataframe[col] = pd.cut(dataframe[col], bins=quartiles, labels=[0.25, 0.50, 0.75, 1], include_lowest=True)

    return dataframe



In [96]:
data_mart_df_V5[get_quarter_cols].head()

,credit_terms_requested,num_live_cards,mortgages_nummortcharges,total_directors_over_time,total_population_2011,rural_population_2011
7492,14,1.0,0.0,3.0,598376.0,73.125000
7493,14,6.0,0.0,2.0,1317788.0,29.484375
7495,14,3.0,1.0,3.0,971888.0,47.000000
7496,14,7.0,0.0,2.0,1132390.0,19.015625
7497,14,9.0,1.0,2.0,1393587.0,33.906250


In [97]:
# Convert 'get_quarter_cols' features to quartiles

data_mart_df_V6 = convert_to_quartiles(data_mart_df_V5, get_quarter_cols)
data_mart_df_V6[get_quarter_cols].head()

,credit_terms_requested,num_live_cards,mortgages_nummortcharges,total_directors_over_time,total_population_2011,rural_population_2011
7492,0.25,0.25,0.25,0.50,0.25,1.00
7493,0.25,0.75,0.25,0.25,1.00,0.25
7495,0.25,0.50,0.75,0.50,0.50,0.50
7496,0.25,1.00,0.25,0.25,1.00,0.25
7497,0.25,1.00,0.75,0.25,1.00,0.25


In [98]:
data_mart_df_V6.credit_terms_requested.value_counts()

credit_terms_requested
0.25    30659
0.75    24272
0.5         0
1.0         0
Name: count, dtype: int64

In [99]:
data_mart_df_V6.mortgages_nummortcharges.value_counts()

mortgages_nummortcharges
0.25    27632
0.75    19497
1.0      7802
0.5         0
Name: count, dtype: int64

In [100]:
# Method for normalization to numeric columns
def normalize_data(dataframe, columns):

    normalized_dataframe = dataframe.copy()

    # Initialize MinMaxScaler
    scaler = MinMaxScaler()

    # Check if the specified columns exist in the DataFrame
    existing_columns = [col for col in columns if col in normalized_dataframe.columns]

    if existing_columns:
        # Fit and transform the specified columns using MinMaxScaler
        normalized_dataframe[existing_columns] = scaler.fit_transform(normalized_dataframe[existing_columns])

    return normalized_dataframe

In [101]:
data_mart_df_V7 = normalize_data(data_mart_df_V6, get_scaler_cols)
data_mart_df_V7[get_scaler_cols].head()

,cash_flow_score,credit_limit_requested,credit_limit_assigned,transaction_diversity,start_date_diff,last_transaction_date_diff,customer_activity_duration,tenure,commercialdelphicreditlimit,commercialdelphiscore,fuel_sites_monthly_avg,total_fee_charge_monthly_avg,number_of_distinct_fees_monthly_avg,sic_code,pence_per_litre,ruc11cd,tenure_months,age,average_age_of_directors,total_litres_monthly_avg,non_dd_litres_monthly_avg,dd_litres_monthly_avg,total_transactions_fuel_monthly_avg,total_transactions_non_visa_monthly_avg,total_transactions_visa_monthly_avg,fuel_spend_monthly_avg,non_fuel_spend_non_visa_monthly_avg,non_fuel_spend_visa_monthly_avg,total_ev_transactions_monthly_avg,total_ev_energy_charge_per_kwh_monthly_avg,total_ev_session_charge_per_minute_monthly_avg,ev_spend_monthly_avg,processed_ruc11,processed_section,processed_uk_region,processed_division
7492,0.0825,0.000096,0.000185,0.333333,1.0,0.264618,0.876021,0.875666,0.000664,0.658750,0.000000,0.955287,0.2,0.827999,0.000089,0.00,0.870968,0.061644,0.305882,0.000381,0.000427,0.000000,0.000023,0.0,0.0,0.000037,0.000185,0.0,0.0,0.004122,0.0,0.056054,0.25,0.190476,0.416667,0.647727
7493,0.2175,0.000385,0.000593,0.666667,1.0,0.004498,0.999290,0.999290,0.002261,0.909375,0.002605,0.956371,0.4,0.432808,0.000086,0.25,1.000000,0.075342,0.294118,0.000874,0.000977,0.000019,0.000391,0.0,0.0,0.000586,0.000185,0.0,0.0,0.004122,0.0,0.056054,1.00,0.333333,0.750000,0.875000
7495,0.0825,0.000067,0.000185,0.333333,1.0,0.621439,0.706927,0.706927,0.000133,0.805625,0.000186,0.955339,0.2,0.531525,0.000087,0.25,0.709677,0.075342,0.352941,0.000367,0.000411,0.000000,0.000046,0.0,0.0,0.000020,0.000185,0.0,0.0,0.004122,0.0,0.056054,0.00,0.000000,0.000000,0.000000
7496,0.0825,0.000231,0.000370,0.666667,1.0,0.012744,0.995382,0.995027,0.000517,0.828125,0.001488,0.956123,0.6,0.708057,0.000090,1.00,0.989247,0.054795,0.352941,0.000538,0.000602,0.000005,0.000207,0.0,0.0,0.000217,0.000185,0.0,0.0,0.004122,0.0,0.056054,0.75,0.714286,0.166667,0.102273
7497,0.0775,0.000394,0.000630,0.333333,1.0,0.011994,0.995737,0.995737,0.001178,0.690625,0.002419,0.955473,0.2,0.427055,0.000089,0.25,0.989247,0.082192,0.494118,0.000754,0.000843,0.000019,0.000368,0.0,0.0,0.000475,0.000185,0.0,0.0,0.004122,0.0,0.056054,1.00,0.333333,0.083333,0.875000


In [103]:
data_mart_df_V7.numberofccjsinlast2years.value_counts()

numberofccjsinlast2years
0.000000    49828
1.000000      459
0.500000      275
0.333252      208
0.250000      175
            ...  
1.687500        1
1.217773        1
1.368164        1
0.958496        1
6.667969        1
Name: count, Length: 460, dtype: int64

In [104]:
# Create bins for 'numberofccjsinlast2years'
bins = [-1,
        data_mart_df_V7['numberofccjsinlast2years'].min(),
        data_mart_df_V7['numberofccjsinlast2years'].astype(int).mean(),
        data_mart_df_V7['numberofccjsinlast2years'].astype(int).mean() + data_mart_df_V7['numberofccjsinlast2years'].astype(int).std(),
        data_mart_df_V7['numberofccjsinlast2years'].max()]

# Split dataset acording to the bins
bins_df = pd.cut(data_mart_df_V7['numberofccjsinlast2years'], bins=bins)

# Create groups with bins
data_shape = data_mart_df_V7.groupby(bins_df).size().reset_index(name='count')

print("Bins and sizes:")
print(data_shape)

Bins and sizes:
  numberofccjsinlast2years  count
0              (-1.0, 0.0]  49828
1            (0.0, 0.0301]      5
2          (0.0301, 0.315]   2039
3            (0.315, 24.0]   3059


In [105]:
data_mart_df_V7['numberofccjsinlast2years'].head(10)

7492    0.000000
7493    0.000000
7495    0.000000
7496    0.000000
7497    0.035706
7498    0.000000
7499    0.000000
7501    0.000000
7503    0.000000
7504    0.000000
Name: numberofccjsinlast2years, dtype: float16

In [106]:
numberofccjsinlast2years_bin_ranges = [data_mart_df_V7['numberofccjsinlast2years'].min(),
                                       data_mart_df_V7['numberofccjsinlast2years'].astype(int).mean(),
                                       data_mart_df_V7['numberofccjsinlast2years'].astype(int).mean() + data_mart_df_V7['numberofccjsinlast2years'].astype(int).std(),
                                       data_mart_df_V7['numberofccjsinlast2years'].max()]
data_mart_df_V7['numberofccjsinlast2years'] = pd.cut(data_mart_df_V7['numberofccjsinlast2years'],
                              bins=numberofccjsinlast2years_bin_ranges,
                              include_lowest=True,
                              duplicates='drop').cat.codes
data_mart_df_V7['numberofccjsinlast2years'].head(10)

7492    0
7493    0
7495    0
7496    0
7497    1
7498    0
7499    0
7501    0
7503    0
7504    0
Name: numberofccjsinlast2years, dtype: int8

In [107]:
data_mart_df_V7['numberofccjsinlast2years'].value_counts()

numberofccjsinlast2years
0    49833
2     3059
1     2039
Name: count, dtype: int64

In [108]:
data_mart_df_V7.shape

(54931, 125)

In [109]:
data_mart_df_V7.iloc[:, 80:].head()

,companycategory_cat_Private_Unlimited_Company,companycategory_cat_Public_Limited_Company,uk_region_cat_East_England,uk_region_cat_Greater_London,uk_region_cat_Isle_of_Man,uk_region_cat_Non-geographic,uk_region_cat_North_East,uk_region_cat_North_West,uk_region_cat_Northern_Ireland,uk_region_cat_Other,uk_region_cat_Scotland,uk_region_cat_South_East,uk_region_cat_South_West,uk_region_cat_Wales,uk_region_cat_West_Midlands,country_cat_England,country_cat_Isle_of_Man,country_cat_Other,country_cat_Scotland,country_cat_Wales,county_cat_Other,county_cat_cumbria,county_cat_devon,county_cat_essex,county_cat_gloucestershire,county_cat_hampshire,county_cat_hertfordshire,county_cat_kent,county_cat_lancashire,county_cat_norfolk,county_cat_north_yorkshire,county_cat_nottinghamshire,county_cat_oxfordshire,county_cat_somerset,county_cat_staffordshire,county_cat_suffolk,county_cat_surrey,county_cat_warwickshire,county_cat_west_sussex,county_cat_worcestershire,ruc11_cat_Largely_Rural,ruc11_cat_Other,ruc11_cat_Urban_with_City_and_Town,ruc11_cat_Urban_with_Major_Conurbation,ruc11_cat_Urban_with_Significant_Rural
7492,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
7493,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7495,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
7496,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
7497,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [110]:
do_not_use_cols

['total_spend_monthly_avg', 'total_transactions_monthly_avg', 'churned']

In [111]:
data_mart_df_V7[['processed_ruc11',	'processed_section',	'processed_uk_region',	'processed_division']].head()


,processed_ruc11,processed_section,processed_uk_region,processed_division
7492,0.25,0.190476,0.416667,0.647727
7493,1.00,0.333333,0.750000,0.875000
7495,0.00,0.000000,0.000000,0.000000
7496,0.75,0.714286,0.166667,0.102273
7497,1.00,0.333333,0.083333,0.875000


In [112]:
data_mart_df_V7[['toplevelcustomerid', 'client_account_number', 'start_date', 'start_date_diff', 'last_transaction_date', 'last_transaction_date_diff']].head()

,toplevelcustomerid,client_account_number,start_date,start_date_diff,last_transaction_date,last_transaction_date_diff
7492,132007,AS60126801,2016-06-23,1.0,2023-03-28,0.264618
7493,132008,AS60126802,2016-06-23,1.0,2024-03-09,0.004498
7495,132026,AS60126820,2016-06-23,1.0,2021-12-07,0.621439
7496,132038,AS60126832,2016-06-23,1.0,2024-02-27,0.012744
7497,132044,AS60126838,2016-06-23,1.0,2024-02-28,0.011994


In [113]:
data_mart_df_V7.last_transaction_date.max()

Timestamp('2024-03-15 00:00:00')

In [114]:
data_mart_df_V7.total_rev_monthly_avg

7492      15
7493     150
7495       7
7496      86
7497     165
        ... 
89392      0
89396      0
89403      0
89404      0
89407      0
Name: total_rev_monthly_avg, Length: 54931, dtype: int32

In [115]:
train_df = data_mart_df_V7[data_mart_df_V7.last_transaction_date < '2024-03-09']
test_df = data_mart_df_V7[data_mart_df_V7.last_transaction_date >= '2024-03-09']
train_df.shape, test_df.shape

((43192, 125), (11739, 125))

In [116]:
train_df.shape, test_df.shape

((43192, 125), (11739, 125))

In [117]:
result_df = test_df[['toplevelcustomerid', 'client_account_number',
                     'start_date', 'last_transaction_date',
                     'total_rev_monthly_avg']]
result_df.shape

(11739, 5)

In [118]:
do_not_use_cols

['total_spend_monthly_avg', 'total_transactions_monthly_avg', 'churned']

In [119]:
train_df.drop(columns=['toplevelcustomerid', 'client_account_number',
                     'start_date', 'last_transaction_date',
                     'total_spend_monthly_avg', 'total_transactions_monthly_avg', 'churned'], inplace=True)
test_df.drop(columns=['toplevelcustomerid', 'client_account_number',
                     'start_date', 'last_transaction_date',
                     'total_spend_monthly_avg', 'total_transactions_monthly_avg', 'churned'], inplace=True)

<ipython-input-119-bad5e01cbbff>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df.drop(columns=['toplevelcustomerid', 'client_account_number',
<ipython-input-119-bad5e01cbbff>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df.drop(columns=['toplevelcustomerid', 'client_account_number',


In [120]:
# Defining a method to move target data to the end of the dataframe
def move_target_to_end(df, target_column):

    # Ensure the target column is in the DataFrame
    if target_column not in df.columns:
        print(f"Error: Target column '{target_column}' not found in the DataFrame.")
        return df

    # Reorder columns to move the target column to the end
    new_order = [col for col in df.columns if col != target_column] + [target_column]
    df = df[new_order]

    return df

In [121]:
# Move target column to the end of list
train_df = move_target_to_end(train_df, 'total_rev_monthly_avg')
test_df = move_target_to_end(test_df, 'total_rev_monthly_avg')

In [122]:
# Split train and test datas for input and output
y_train = train_df['total_rev_monthly_avg'].copy() #features (independent variables)
X_train = train_df.drop(columns=['total_rev_monthly_avg']) #features (independent variables)
y_test = test_df['total_rev_monthly_avg'].copy() #features (independent variables)
X_test = test_df.drop(columns=['total_rev_monthly_avg']) #features (independent variables)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((43192, 117), (43192,), (11739, 117), (11739,))

In [123]:
X_train = reduce_mem_usage(X_train)
X_test = reduce_mem_usage(X_test)

Mem. usage decreased to  6.76 Mb (30.5% reduction)
Mem. usage decreased to  1.84 Mb (30.5% reduction)


In [124]:
result_df.to_csv("results.csv", index=False)

result_df.head()

,toplevelcustomerid,client_account_number,start_date,last_transaction_date,total_rev_monthly_avg
7493,132008,AS60126802,2016-06-23,2024-03-09,150
7501,132059,AS60126853,2016-06-24,2024-03-12,37
7504,132068,AS60126862,2016-06-24,2024-03-12,16
7525,132138,AS60126932,2016-06-27,2024-03-12,282
7530,132156,AS60126950,2016-06-28,2024-03-10,49


In [125]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((43192, 117), (43192,), (11739, 117), (11739,))

##XGB Regressor

In [127]:
# Define custom MAPE scorer
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [128]:
# Define XGBRegressor

%%time
xgb = XGBRegressor(enable_categorical=True)

# Define parameter grid for grid search
param_grid = { 'gamma' : [0.3, 1], 'learning_rate' : [0.001, 0.01, 0.1],
              'max_depth' : [4, 8, 12],
              'n_estimators' : [10, 50, 100],
              'nthread' : [-1], 'reg_alpha' : [1], 'reg_lambda' : [1], 'seed' : [10]
              }

# Perform grid search with time series cross-validation
#tscv = TimeSeriesSplit(n_splits=-1)
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=5, verbose=True)
grid_search.fit(X_train, y_train, eval_set=[(X_train, y_train)], early_stopping_rounds=50)


Fitting 5 folds for each of 54 candidates, totalling 270 fits
[0]	validation_0-rmse:252.44468
[1]	validation_0-rmse:252.35866


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[2]	validation_0-rmse:252.27279
[3]	validation_0-rmse:252.18703
[4]	validation_0-rmse:252.10140
[5]	validation_0-rmse:252.01588
[6]	validation_0-rmse:251.93050
[7]	validation_0-rmse:251.84523
[8]	validation_0-rmse:251.76008
[9]	validation_0-rmse:251.67506
[0]	validation_0-rmse:252.28622
[1]	validation_0-rmse:252.15845
[2]	validation_0-rmse:252.03080


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-rmse:251.90327
[4]	validation_0-rmse:251.77586
[5]	validation_0-rmse:251.64858
[6]	validation_0-rmse:251.52143
[7]	validation_0-rmse:251.39440
[8]	validation_0-rmse:251.26750
[9]	validation_0-rmse:251.14071
[0]	validation_0-rmse:252.28084
[1]	validation_0-rmse:252.14345
[2]	validation_0-rmse:252.00623


...

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[2]	validation_0-rmse:207.62996
[3]	validation_0-rmse:194.72120
[4]	validation_0-rmse:183.11357
[5]	validation_0-rmse:172.49760
[6]	validation_0-rmse:162.99669
[7]	validation_0-rmse:154.01457
[8]	validation_0-rmse:145.90676
[9]	validation_0-rmse:138.29056
[10]	validation_0-rmse:131.43087
[11]	validation_0-rmse:124.73796
[12]	validation_0-rmse:118.73829
[13]	validation_0-rmse:113.31724
[14]	validation_0-rmse:107.91421
[15]	validation_0-rmse:103.16655
[16]	validation_0-rmse:98.80121
[17]	validation_0-rmse:94.84993
[18]	validation_0-rmse:91.12878
[19]	validation_0-rmse:87.78367
[20]	validation_0-rmse:84.65858
[21]	validation_0-rmse:81.81162
[22]	validation_0-rmse:79.11161
[23]	validation_0-rmse:76.64491
[24]	validation_0-rmse:74.37058
[25]	validation_0-rmse:72.27423
[26]	validation_0-rmse:70.37288
[27]	validation_0-rmse:68.66072
[28]	validation_0-rmse:67.09893
[29]	validation_0-rmse:65.64798
[30]	validation_0-rmse:64.30034
[31]	validation_0-rmse:63.04015
[32]	validation_0-rmse:61.94211
[3

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[3]	validation_0-rmse:191.96212
[4]	validation_0-rmse:179.84482
[5]	validation_0-rmse:168.60986
[6]	validation_0-rmse:158.26108
[7]	validation_0-rmse:148.66455
[8]	validation_0-rmse:139.79869
[9]	validation_0-rmse:131.58477
CPU times: user 18min 12s, sys: 9.93 s, total: 18min 22s
Wall time: 18min 45s


GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=True, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, ma...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'gamma': [0.3, 1], 'learning_rate': [0.001, 0.01, 0.1],
                         'max_depth': [4, 8, 12], 'n_estimators': [10, 50, 100],
                         'nthread': [-1], 'reg_alpha': [1], 'reg_lambda': [1],
                         'seed': [10]},
             verbose=True)

In [129]:
grid_search.best_estimator_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             gamma=1, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=12, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None, nthread=-1,
             num_parallel_tree=None, ...)

In [130]:
grid_search.best_params_

{'gamma': 1,
 'learning_rate': 0.1,
 'max_depth': 12,
 'n_estimators': 10,
 'nthread': -1,
 'reg_alpha': 1,
 'reg_lambda': 1,
 'seed': 10}

In [131]:
grid_search.best_score_

0.4178522147007306

In [132]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate absolute percentage error
    absolute_percentage_error = np.abs((y_true - y_pred) / y_true)

    # Handle division by zero (replace with zero)
    absolute_percentage_error[np.isinf(absolute_percentage_error)] = 0

    # Calculate mean absolute percentage error
    mape = np.mean(absolute_percentage_error) * 100

    return mape


In [133]:
%%time
# Predict with X_test data
y_pred = grid_search.predict(X_test)

from sklearn.metrics import mean_squared_error

# Calculate RMSE and MAPE
rmse = np.sqrt(mean_squared_error(y_test.astype(np.int32), y_pred.astype(np.int32)))
mape = mean_absolute_percentage_error(y_test.astype(np.int32), y_pred.astype(np.int32))

print("RMSE:", rmse)
print("MAPE:", mape)


RMSE: 406.07096077043053
MAPE: 136.53032854380507
CPU times: user 49.8 ms, sys: 13 µs, total: 49.8 ms
Wall time: 63.7 ms


<ipython-input-132-14d96e724ddc>:6: RuntimeWarning: divide by zero encountered in divide
  absolute_percentage_error = np.abs((y_true - y_pred) / y_true)


In [134]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11739 entries, 7493 to 89407
Data columns (total 5 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   toplevelcustomerid     11739 non-null  int32         
 1   client_account_number  11739 non-null  object        
 2   start_date             11739 non-null  datetime64[ns]
 3   last_transaction_date  11739 non-null  datetime64[ns]
 4   total_rev_monthly_avg  11739 non-null  int32         
dtypes: datetime64[ns](2), int32(2), object(1)
memory usage: 458.6+ KB


In [135]:
result_df['XGB_Preds'] = np.round(y_pred).astype(int)
result_df.to_csv("results.csv", index=False)

result_df.head(10)

,toplevelcustomerid,client_account_number,start_date,last_transaction_date,total_rev_monthly_avg,XGB_Preds
7493,132008,AS60126802,2016-06-23,2024-03-09,150,120
7501,132059,AS60126853,2016-06-24,2024-03-12,37,61
7504,132068,AS60126862,2016-06-24,2024-03-12,16,33
7525,132138,AS60126932,2016-06-27,2024-03-12,282,209
7530,132156,AS60126950,2016-06-28,2024-03-10,49,59
7531,132158,AS60126952,2016-06-28,2024-03-12,240,165
7539,132184,AS60126978,2016-06-28,2024-03-12,544,348
7543,132205,AS60126999,2016-06-29,2024-03-10,48,56
7552,132235,AS60127029,2016-06-29,2024-03-12,138,108
7557,132258,AS60127052,2016-06-30,2024-03-11,22,29


##Recurrent Neural Network

In [146]:
import torch
import torch.nn as nn

# Define training parameters
lr = 0.1
dropout_rate = 0.2
num_epochs = 100

# Prepare the data
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)

# Convert y_train Series to a NumPy array and reshape it
y_train_array = np.array(y_train).reshape(-1, 1)
# Convert the NumPy array to a PyTorch tensor
y_train_tensor = torch.tensor(y_train_array, dtype=torch.float32)

# Convert y_train Series to a NumPy array and reshape it
y_test_array = np.array(y_test).reshape(-1, 1)
# Convert the NumPy array to a PyTorch tensor
y_test_tensor = torch.tensor(y_test_array, dtype=torch.float32)

# Define the RNN Model
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.hidden_size)  # Reset the initial hidden state to zero
        out, _ = self.rnn(x, h0)  # Apply the RNN layer
        #out = self.Dropout(out)
        #out = self.elu(out)
        out = self.fc(out[:, -1, :])  # Apply the fully connected layer by taking the output of the last time step
        return out

# Model parameters
input_size = X_train.shape[1]  # Input size
hidden_size = 512  # Hidden layer size
output_size = 1  # Output size (prediction)

# Create the model
model = RNN(input_size, hidden_size, output_size)


In [148]:
%%time
# Reshaping the training and test data
X_train_tensor = X_train_tensor.view(-1, 1, input_size)
X_test_tensor = X_test_tensor.view(-1, 1, input_size)

# Define optimization and loss function
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
#num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}')

# Test
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    test_loss = criterion(outputs.squeeze(), y_test_tensor)
print("Test Loss:", test_loss.item())


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([43192, 1])) that is different to the input size (torch.Size([43192])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/100, Loss: 67634.8672
Epoch 2/100, Loss: 67273.9062
Epoch 3/100, Loss: 66840.1250
Epoch 4/100, Loss: 66342.3984
Epoch 5/100, Loss: 65830.2266
Epoch 6/100, Loss: 65345.1992
Epoch 7/100, Loss: 64914.2461
Epoch 8/100, Loss: 64550.0000
Epoch 9/100, Loss: 64255.7695
Epoch 10/100, Loss: 64030.0742
Epoch 11/100, Loss: 63869.0859
Epoch 12/100, Loss: 63767.1641
Epoch 13/100, Loss: 63717.0547
Epoch 14/100, Loss: 63709.4688
Epoch 15/100, Loss: 63733.3398
Epoch 16/100, Loss: 63776.5117
Epoch 17/100, Loss: 63826.9180
Epoch 18/100, Loss: 63874.1680
Epoch 19/100, Loss: 63910.6406
Epoch 20/100, Loss: 63932.0039
Epoch 21/100, Loss: 63937.0312
Epoch 22/100, Loss: 63926.8672
Epoch 23/100, Loss: 63904.4219
Epoch 24/100, Loss: 63873.4141
Epoch 25/100, Loss: 63838.0625
Epoch 26/100, Loss: 63802.4180
Epoch 27/100, Loss: 63769.9180
Epoch 28/100, Loss: 63743.1289
Epoch 29/100, Loss: 63723.6602
Epoch 30/100, Loss: 63711.9727
Epoch 31/100, Loss: 63707.7031
Epoch 32/100, Loss: 63709.5430
Epoch 33/100, Los

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([11739, 1])) that is different to the input size (torch.Size([11739])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss: 307172.625
CPU times: user 9min 12s, sys: 8min 12s, total: 17min 25s
Wall time: 17min 26s


In [152]:
# Making predictions on the test data
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = outputs.squeeze().numpy()
y_test_array = y_test.values

# Comparing predictions with the actual values
for i in range(30):  # Inspecting the first 30 examples
    print("Actual Value:", y_test_array[i], "\tPrediction:", predictions[i])


Gerçek Değer: 150 	Tahmin: 62.680874
Gerçek Değer: 37 	Tahmin: 62.69168
Gerçek Değer: 16 	Tahmin: 62.668457
Gerçek Değer: 282 	Tahmin: 62.682278
Gerçek Değer: 49 	Tahmin: 62.67927
Gerçek Değer: 240 	Tahmin: 62.678497
Gerçek Değer: 544 	Tahmin: 62.680332
Gerçek Değer: 48 	Tahmin: 62.636456
Gerçek Değer: 138 	Tahmin: 62.695904
Gerçek Değer: 22 	Tahmin: 62.68218
Gerçek Değer: 505 	Tahmin: 62.688766
Gerçek Değer: 452 	Tahmin: 62.693913
Gerçek Değer: 299 	Tahmin: 62.683098
Gerçek Değer: 417 	Tahmin: 62.68726
Gerçek Değer: 244 	Tahmin: 62.682392
Gerçek Değer: 53 	Tahmin: 62.6235
Gerçek Değer: 121 	Tahmin: 62.646458
Gerçek Değer: 140 	Tahmin: 62.67101
Gerçek Değer: 860 	Tahmin: 62.682774
Gerçek Değer: 18 	Tahmin: 62.67542
Gerçek Değer: 9 	Tahmin: 62.61546
Gerçek Değer: 9 	Tahmin: 62.662853
Gerçek Değer: 34318 	Tahmin: 62.69831
Gerçek Değer: 37 	Tahmin: 62.674107
Gerçek Değer: 120 	Tahmin: 62.68176
Gerçek Değer: 482 	Tahmin: 62.669094
Gerçek Değer: 16 	Tahmin: 62.682335
Gerçek Değer: 70 	Tahmi

In [154]:
# Calculate RMSE and MAPE
rmse = np.sqrt(mean_squared_error(y_test_array.astype(np.int32), predictions.astype(np.int32)))
mape = mean_absolute_percentage_error(y_test_array.astype(np.int32), predictions.astype(np.int32))

print("RMSE:", rmse)
print("MAPE:", mape)


RMSE: 554.3000498614708
MAPE: 360.8442949472074


<ipython-input-132-14d96e724ddc>:6: RuntimeWarning: divide by zero encountered in divide
  absolute_percentage_error = np.abs((y_true - y_pred) / y_true)


In [155]:
result_df['RNN_Preds'] = np.round(predictions).astype(int)
result_df.to_csv("results.csv", index=False)

result_df.head(10)

,toplevelcustomerid,client_account_number,start_date,last_transaction_date,total_rev_monthly_avg,XGB_Preds,RNN_Preds
7493,132008,AS60126802,2016-06-23,2024-03-09,150,120,63
7501,132059,AS60126853,2016-06-24,2024-03-12,37,61,63
7504,132068,AS60126862,2016-06-24,2024-03-12,16,33,63
7525,132138,AS60126932,2016-06-27,2024-03-12,282,209,63
7530,132156,AS60126950,2016-06-28,2024-03-10,49,59,63
7531,132158,AS60126952,2016-06-28,2024-03-12,240,165,63
7539,132184,AS60126978,2016-06-28,2024-03-12,544,348,63
7543,132205,AS60126999,2016-06-29,2024-03-10,48,56,63
7552,132235,AS60127029,2016-06-29,2024-03-12,138,108,63
7557,132258,AS60127052,2016-06-30,2024-03-11,22,29,63
